In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [29]:
KM_PER_NM = 1.852
def calculate_ellipse_area(r_ne, r_se, r_sw, r_nw):
	if any(np.isnan(x) for x in [r_ne, r_se, r_sw, r_nw]):
		return np.nan
	a = KM_PER_NM * (r_ne + r_sw)
	b = KM_PER_NM * (r_nw + r_se)
	area = np.pi * a * b
	return np.nan if area == 0 else area

In [30]:
def calculate_circle_area(r):
	if np.isnan(r) or r == 0:
		return np.nan
	return np.pi * (KM_PER_NM * r) ** 2

In [31]:
df = pd.read_csv("../data/raw/hurdat2_cleaned.csv")

In [32]:
speeds = [34, 50, 64]
for v in speeds:
	df[f'size_{v}kt_sqkm'] = df.apply(axis = 1, func = lambda row: calculate_ellipse_area(row[f'r_{v}kt_ne'], row[f'r_{v}kt_se'], row[f'r_{v}kt_sw'], row[f'r_{v}kt_nw']))

df['size_max_sqkm'] = df.apply(axis = 1, func = lambda row: calculate_circle_area(row['r_max_sus']))

In [33]:
gb = df.groupby('code').agg(
	{
		'size_34kt_sqkm': 'max',
		'size_50kt_sqkm': 'max',
		'size_64kt_sqkm': 'max',
		'size_max_sqkm': 'max'
	}
)
gb.reset_index(inplace = True)

In [34]:
lst = ['size_34kt_sqkm', 'size_50kt_sqkm', 'size_64kt_sqkm', 'size_max_sqkm']
gb.describe()

,size_34kt_sqkm,size_50kt_sqkm,size_64kt_sqkm,size_max_sqkm
count,3.430000e+02,2.340000e+02,160.000000,2.700000e+02
mean,1.104759e+06,2.790266e+05,80540.774086,5.105326e+04
std,1.863165e+06,3.927222e+05,104750.776762,1.592800e+05
min,4.310144e+03,2.693840e+02,1077.536121,2.693840e+02
25%,1.293043e+05,3.879130e+04,12930.433451,2.424456e+03
50%,3.879130e+05,1.018272e+05,35558.691989,6.734601e+03
75%,1.280113e+06,3.480442e+05,111457.642505,3.879130e+04
max,1.675784e+07,2.143219e+06,568939.071831,1.724058e+06


In [35]:
gb_filled = gb.fillna(0) # can't use nan in max

In [36]:
gb_filled['biggest'] = gb_filled[lst].max(axis=1)
len(gb_filled[gb_filled['biggest'] != 0])

509

In [37]:
results = gb_filled[['code','biggest']]
results.columns = ['code','peak_size_sqkm']
results

,code,peak_size_sqkm
0,AL011851,0.000000e+00
1,AL011852,1.077536e+03
2,AL011853,0.000000e+00
3,AL011854,0.000000e+00
4,AL011855,0.000000e+00
...,...,...
1986,AL301969,0.000000e+00
1987,AL302005,5.689391e+05
1988,AL302020,8.350905e+05
1989,AL312005,4.714221e+05


In [43]:
results.loc[results.peak_size_sqkm == 0,'peak_size_sqkm'] = np.nan
results

,code,peak_size_sqkm
0,AL011851,NaN
1,AL011852,1.077536e+03
2,AL011853,NaN
3,AL011854,NaN
4,AL011855,NaN
...,...,...
1986,AL301969,NaN
1987,AL302005,5.689391e+05
1988,AL302020,8.350905e+05
1989,AL312005,4.714221e+05


In [46]:
results.to_csv('../data/processed/outcomes/size.csv', index = False)